# Module 2: AST 분석 및 시각화

이 노트북에서는 Python의 **AST(Abstract Syntax Tree)**를 다룹니다.

## 목차
1. [AST란?](#1-ast란)
2. [ast 모듈 기본 사용법](#2-ast-모듈-기본-사용법)
3. [AST 노드 유형 학습](#3-ast-노드-유형-학습)
4. [실습 1: 간단한 코드의 AST 분석](#4-실습-1-간단한-코드의-ast-분석)
5. [실습 2: 함수 정의 AST](#5-실습-2-함수-정의-ast)
6. [실습 3: ast_visualizer 도구 사용](#6-실습-3-ast_visualizer-도구-사용)
7. [실습 4: AST 변환 (NodeTransformer)](#7-실습-4-ast-변환-nodetransformer)
8. [연습 문제](#8-연습-문제)

## 1. AST란?

**AST(Abstract Syntax Tree, 추상 구문 트리)**는 소스 코드의 구조를 트리 형태로 표현한 것입니다.

### 왜 AST가 중요할까요?

- **코드 분석**: 코드의 구조를 이해하고 분석할 수 있습니다
- **코드 변환**: 코드를 자동으로 수정하거나 변환할 수 있습니다
- **정적 분석**: 버그나 코드 스멜을 찾을 수 있습니다
- **코드 생성**: 메타프로그래밍과 코드 생성에 사용됩니다

### AST vs 일반 텍스트

```python
# 소스 코드 (텍스트)
x = 1 + 2

# AST (트리 구조)
# Module
# └── Assign
#     ├── targets: [Name(id='x')]
#     └── value: BinOp
#         ├── left: Constant(value=1)
#         ├── op: Add()
#         └── right: Constant(value=2)
```

## 2. ast 모듈 기본 사용법

Python의 `ast` 모듈은 표준 라이브러리에 포함되어 있어 별도 설치가 필요 없습니다.

In [ ]:
import ast

# 간단한 코드 파싱
code = "x = 1 + 2"
tree = ast.parse(code)

print("AST 객체 타입:", type(tree))
print("\nAST 구조 (dump):")
print(ast.dump(tree, indent=2))

### ast.parse()

`ast.parse(source)`는 문자열 형태의 Python 코드를 파싱하여 AST 트리를 반환합니다.

### ast.dump()

`ast.dump(node, indent=2)`는 AST 노드를 사람이 읽을 수 있는 문자열로 변환합니다.
`indent` 옵션을 사용하면 보기 좋게 들여쓰기됩니다.

In [ ]:
# 다양한 코드 파싱 예제
examples = [
    "print('Hello, World!')",
    "x = [1, 2, 3]",
    "for i in range(10):\n    print(i)",
    "def add(a, b):\n    return a + b",
]

for i, code in enumerate(examples, 1):
    print(f"\n{'='*50}")
    print(f"예제 {i}: {code[:30]}...")
    print(f"{'='*50}")
    tree = ast.parse(code)
    print(ast.dump(tree, indent=2)[:500] + "..." if len(ast.dump(tree)) > 500 else ast.dump(tree, indent=2))

## 3. AST 노드 유형 학습

AST 노드는 크게 **문장(Statement)**과 **표현식(Expression)**으로 나뉩니다.

### 3.1 문장 (Statement) 노드

| 노드 타입 | 설명 | 예시 |
|-----------|------|------|
| `FunctionDef` | 함수 정의 | `def foo(): ...` |
| `ClassDef` | 클래스 정의 | `class Foo: ...` |
| `If` | 조걸문 | `if x > 0: ...` |
| `For` | for 루프 | `for i in range(10): ...` |
| `While` | while 루프 | `while x < 10: ...` |
| `Return` | 반환문 | `return x + 1` |
| `Assign` | 할당문 | `x = 1` |
| `AugAssign` | 복합 할당 | `x += 1` |
| `Import` | 임포트 | `import os` |
| `ImportFrom` | from 임포트 | `from os import path` |

In [ ]:
# 문장 노드 예제
statements = {
    'FunctionDef': '''
def greet(name):
    return f"Hello, {name}!"
''',
    'If': '''
if x > 0:
    print("positive")
else:
    print("non-positive")
''',
    'For': '''
for i in range(5):
    print(i)
''',
    'Return': '''
return x + y
''',
}

for stmt_type, code in statements.items():
    print(f"\n{'='*50}")
    print(f"{stmt_type} 예제:")
    print(f"{'='*50}")
    tree = ast.parse(code)
    # 첫 번째 문장의 타입 확인
    first_stmt = tree.body[0]
    print(f"첫 번째 문장 타입: {type(first_stmt).__name__}")
    print(f"\nAST 구조:")
    print(ast.dump(first_stmt, indent=2))

### 3.2 표현식 (Expression) 노드

| 노드 타입 | 설명 | 예시 |
|-----------|------|------|
| `BinOp` | 이항 연산 | `a + b`, `x * y` |
| `UnaryOp` | 단항 연산 | `-x`, `not flag` |
| `BoolOp` | 논리 연산 | `a and b`, `x or y` |
| `Compare` | 비교 연산 | `x > 0`, `a == b` |
| `Call` | 함수 호출 | `print("hi")` |
| `Name` | 변수 이름 | `x`, `foo` |
| `Constant` | 상수 | `42`, `"hello"` |
| `Attribute` | 속성 접근 | `obj.method` |
| `Subscript` | 첨자 접근 | `lst[0]`, `dict["key"]` |

In [ ]:
# 표현식 노드 예제
expressions = {
    'BinOp': '1 + 2',
    'UnaryOp': '-x',
    'BoolOp': 'a and b',
    'Compare': 'x > 0',
    'Call': 'print("hello")',
    'Name': 'x',
    'Constant': '42',
    'Attribute': 'obj.method',
    'Subscript': 'lst[0]',
}

for expr_type, code in expressions.items():
    print(f"\n{'='*40}")
    print(f"{expr_type}: {code}")
    print(f"{'='*40}")
    tree = ast.parse(code, mode='eval')  # 표현식 모드로 파싱
    print(ast.dump(tree, indent=2))

## 4. 실습 1: 간단한 코드의 AST 분석

`x = 1 + 2`라는 간단한 코드의 AST 구조를 분석해봅시다.

In [ ]:
import ast

# 분석할 코드
code = "x = 1 + 2"
tree = ast.parse(code)

print("코드:", code)
print("\n전체 AST 구조:")
print(ast.dump(tree, indent=2))

In [ ]:
# 트리 구조 탐색
print("트리 구조 분석:")
print(f"루트 노드: {type(tree).__name__}")
print(f"  body 길이: {len(tree.body)}")

# 첫 번째 문장 (Assign)
assign_stmt = tree.body[0]
print(f"\n1단계 - 문장:")
print(f"  타입: {type(assign_stmt).__name__}")
print(f"  targets: {assign_stmt.targets}")
print(f"  value: {type(assign_stmt.value).__name__}")

# 할당 대상 (Name)
target = assign_stmt.targets[0]
print(f"\n2단계 - 할당 대상:")
print(f"  타입: {type(target).__name__}")
print(f"  id: {target.id}")

# 값 (BinOp)
binop = assign_stmt.value
print(f"\n3단계 - 값 (BinOp):")
print(f"  타입: {type(binop).__name__}")
print(f"  left: {type(binop.left).__name__} = {binop.left.value}")
print(f"  op: {type(binop.op).__name__}")
print(f"  right: {type(binop.right).__name__} = {binop.right.value}")

### 트리 구조 시각화

```
Module
└── body: [Assign]
    ├── targets: [Name]
    │   └── id: 'x'
    └── value: BinOp
        ├── left: Constant
        │   └── value: 1
        ├── op: Add
        └── right: Constant
            └── value: 2
```

## 5. 실습 2: 함수 정의 AST

함수 정의의 AST 구조를 분석해봅시다.

In [ ]:
# 함수 정의 코드
code = '''
def add(a, b=10):
    result = a + b
    return result
'''

tree = ast.parse(code)
print("전체 AST:")
print(ast.dump(tree, indent=2))

In [ ]:
# 함수 정의 구조 분석
func_def = tree.body[0]

print("함수 정의 분석:")
print(f"{'='*50}")

# 기본 정보
print(f"함수 이름: {func_def.name}")
print(f"함수 타입: {type(func_def).__name__}")

# 인자 (arguments)
args = func_def.args
print(f"\n인자 정보 (arguments):")
print(f"  타입: {type(args).__name__}")
print(f"  args: {[arg.arg for arg in args.args]}")
print(f"  defaults: {args.defaults}")

# 각 인자 상세 정보
print(f"\n  인자 상세:")
for i, arg in enumerate(args.args):
    print(f"    - {arg.arg} (타입: {type(arg).__name__})")

# 함수 본문 (body)
print(f"\n함수 본문 (body):")
print(f"  문장 수: {len(func_def.body)}")
for i, stmt in enumerate(func_def.body):
    print(f"  [{i}] {type(stmt).__name__}")

### 함수 정의 AST 구조

```
FunctionDef
├── name: 'add'           # 함수 이름
├── args: arguments       # 인자 정보
│   ├── args: [arg, arg]  # 위치 인자들
│   ├── defaults: [10]    # 기본값들
│   └── ...               # *args, **kwargs 등
├── body: [Assign, Return]  # 함수 본문
├── decorator_list: []    # 데코레이터
└── returns: None         # 반환 타입 힌트
```

## 6. 실습 3: ast_visualizer 도구 사용

프로젝트에 포함된 `tools/ast_visualizer` 모듈을 사용하여 AST를 시각화할 수 있습니다.

> **참고**: 이미지 생성을 위해서는 `graphviz` 시스템 패키지가 설치되어 있어야 합니다.
> - Ubuntu/Debian: `sudo apt-get install graphviz`
> - macOS: `brew install graphviz`
> - Windows: [graphviz.org](https://graphviz.org/download/)에서 다운로드

In [ ]:
# ast_visualizer 임포트
import sys
sys.path.insert(0, '..')  # 상위 디렉토리에서 tools 접근

try:
    from tools.ast_visualizer import visualize_ast
    print("✓ ast_visualizer 임포트 성공")
except ImportError as e:
    print(f"✗ 임포트 오류: {e}")

In [ ]:
# 시각화 예제 코드
code = '''
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)
'''

# 먼저 AST dump로 확인
tree = ast.parse(code)
print("AST 구조:")
print(ast.dump(tree, indent=2))

In [ ]:
# AST 시각화 시도
import os

# 출력 디렉토리 생성
output_dir = '../outputs/ast'
os.makedirs(output_dir, exist_ok=True)

try:
    output_path = visualize_ast(
        code,
        f'{output_dir}/factorial_ast.png',
        title="Factorial Function AST"
    )
    print(f"✓ AST 이미지 생성 완료: {output_path}")
    
    # 이미지 표시
    from IPython.display import Image, display
    display(Image(filename=output_path))
except Exception as e:
    print(f"✗ 시각화 실패: {e}")
    print("\nℹ️ graphviz가 설치되어 있지 않으면 이미지 생성이 실패합니다.")
    print("   AST 구조는 위의 dump 출력으로 확인할 수 있습니다.")

### visualize_ast 함수 사용법

```python
from tools.ast_visualizer import visualize_ast

# 기본 사용법
visualize_ast(
    code="def foo(): pass",    # Python 코드 문자열
    output="ast.png",          # 출력 파일 경로
    title="My AST"             # 그래프 제목 (선택)
)
```

### 노드 색상 의미

- **파란색 계열**: Module, FunctionDef, ClassDef (정의)
- **주황색 계열**: If, For, While, Return 등 (제어 흐름)
- **녹색 계열**: BinOp, Call, Name 등 (표현식)
- **회색 계열**: Constant, Num, Str 등 (리터럴)

In [ ]:
# 더 복잡한 예제: 클래스 정의
class_code = '''
class Calculator:
    def __init__(self):
        self.result = 0
    
    def add(self, x):
        self.result += x
        return self.result
'''

try:
    output_path = visualize_ast(
        class_code,
        f'{output_dir}/calculator_ast.png',
        title="Calculator Class AST"
    )
    print(f"✓ 클래스 AST 이미지 생성 완료")
    from IPython.display import Image, display
    display(Image(filename=output_path))
except Exception as e:
    print(f"✗ 시각화 실패: {e}")
    # AST 텍스트로 출력
    tree = ast.parse(class_code)
    print("\nAST 텍스트 출력:")
    print(ast.dump(tree, indent=2))

## 7. 실습 4: AST 변환 (NodeTransformer)

AST를 수정하여 코드를 변환할 수 있습니다. `ast.NodeTransformer`를 상속받아 노드를 변환합니다.

### 7.1 기본적인 NodeTransformer

In [ ]:
# 모든 숫자를 두 배로 만드는 Transformer
class DoubleNumbers(ast.NodeTransformer):
    def visit_Constant(self, node):
        if isinstance(node.value, (int, float)):
            # 숫자를 두 배로
            return ast.Constant(value=node.value * 2)
        return node

# 테스트 코드
code = "x = 1 + 2 * 3"
tree = ast.parse(code)

print("원본 코드:", code)
print("\n원본 AST:")
print(ast.dump(tree, indent=2))

# 변환 적용
transformer = DoubleNumbers()
new_tree = transformer.visit(tree)

print("\n변환된 AST:")
print(ast.dump(new_tree, indent=2))

# 변환된 코드 확인
import astor
try:
    new_code = astor.to_source(new_tree)
    print(f"\n변환된 코드: {new_code}")
except ImportError:
    print("\nℹ️ astor 패키지가 설치되어 있지 않아 코드 생성은 생략합니다.")

### 7.2 print() 호출에 시간 로깅 추가

모든 `print()` 호출 앞에 현재 시간을 출력하는 코드를 자동으로 삽입합니다.

In [ ]:
class PrintTimeInjector(ast.NodeTransformer):
    """
    모든 print() 호출 앞에 시간 로깅을 추가하는 Transformer
    """
    
    def visit_Expr(self, node):
        # 표현식 문장 처리
        if isinstance(node.value, ast.Call):
            call = node.value
            if isinstance(call.func, ast.Name) and call.func.id == 'print':
                # print() 호출 발견!
                # 시간 출력 코드 생성: print(f"[HH:MM:SS]", ...)
                time_format = ast.JoinedStr(values=[
                    ast.Constant(value='['),
                    ast.FormattedValue(
                        value=ast.Call(
                            func=ast.Attribute(
                                value=ast.Call(
                                    func=ast.Name(id='__import__', ctx=ast.Load()),
                                    args=[ast.Constant(value='datetime')],
                                    keywords=[]
                                ),
                                attr='datetime',
                                ctx=ast.Load()
                            ),
                            args=[],
                            keywords=[]
                        ),
                        conversion=-1,
                        format_spec=ast.Constant(value='%H:%M:%S')
                    ),
                    ast.Constant(value='] ')
                ])
                
                # 기존 인자 앞에 시간 추가
                new_args = [time_format] + call.args
                call.args = new_args
        return node

# 더 간단한 버전: 직접 코드 문자열로 생성
class SimplePrintTransformer(ast.NodeTransformer):
    """
    print 호출을 감싸는 간단한 Transformer
    """
    
    def visit_Module(self, node):
        # datetime 임포트 추가
        import_node = ast.parse("from datetime import datetime").body[0]
        node.body.insert(0, import_node)
        return self.generic_visit(node)
    
    def visit_Call(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == 'print':
            # print 앞에 시간 출력 추가
            print(f"[DEBUG] print 호출 발견!")
        return self.generic_visit(node)

In [ ]:
# 더 실용적인 예제: print 호출을 로깅 함수로 래핑
code = '''
def greet(name):
    print(f"Hello, {name}!")
    print("Welcome!")

greet("World")
'''

tree = ast.parse(code)
print("원본 코드:")
print(code)

In [ ]:
# 함수 이름을 대문자로 변환하는 Transformer
class UppercaseFunctionNames(ast.NodeTransformer):
    def visit_FunctionDef(self, node):
        # 함수 이름을 대문자로
        node.name = node.name.upper()
        # 자식 노드도 계속 방문
        return self.generic_visit(node)

# 변환 적용
transformer = UppercaseFunctionNames()
new_tree = transformer.visit(ast.parse(code))

print("변환된 AST:")
print(ast.dump(new_tree, indent=2))

# 코드로 변환 (astunparse 사용)
try:
    import astunparse
    new_code = astunparse.unparse(new_tree)
    print("\n변환된 코드:")
    print(new_code)
except ImportError:
    print("\nℹ️ astunparse가 설치되어 있지 않습니다.")
    print("   pip install astunparse 로 설치할 수 있습니다.")

### 7.3 실전 예제: 코드 인스트루먼테이션

함수의 시작과 끝에 로깅을 추가하는 Transformer

In [ ]:
class FunctionLogger(ast.NodeTransformer):
    """
    함수의 시작과 끝에 로깅 코드를 추가하는 Transformer
    """
    
    def visit_FunctionDef(self, node):
        # 원래 함수 이름 저장
        func_name = node.name
        
        # 시작 로그: print(f"[ENTER] 함수이름")
        enter_log = ast.Expr(
            value=ast.Call(
                func=ast.Name(id='print', ctx=ast.Load()),
                args=[
                    ast.JoinedStr(values=[
                        ast.Constant(value='[ENTER] '),
                        ast.Constant(value=func_name)
                    ])
                ],
                keywords=[]
            )
        )
        
        # 종료 로그: print(f"[EXIT] 함수이름")
        exit_log = ast.Expr(
            value=ast.Call(
                func=ast.Name(id='print', ctx=ast.Load()),
                args=[
                    ast.JoinedStr(values=[
                        ast.Constant(value='[EXIT] '),
                        ast.Constant(value=func_name)
                    ])
                ],
                keywords=[]
            )
        )
        
        # 본문 수정: 시작 로그 추가
        node.body.insert(0, enter_log)
        
        # Return 문을 찾아서 앞에 종료 로그 추가
        new_body = []
        for stmt in node.body:
            if isinstance(stmt, ast.Return):
                # Return 앞에 종료 로그
                new_body.append(exit_log)
            new_body.append(stmt)
        
        # Return이 없는 경우 (맨 끝에 종료 로그)
        if not any(isinstance(s, ast.Return) for s in node.body):
            new_body.append(exit_log)
        
        node.body = new_body
        
        return self.generic_visit(node)

# 테스트
test_code = '''
def add(a, b):
    return a + b

def greet(name):
    print(f"Hello, {name}")
'''

tree = ast.parse(test_code)
transformer = FunctionLogger()
new_tree = transformer.visit(tree)

print("변환된 AST:")
print(ast.dump(new_tree, indent=2))

In [ ]:
# 변환된 코드 실행해보기
# compile() 함수로 AST를 컴파일하여 실행할 수 있습니다

# AST를 코드 객체로 컴파일
code_obj = compile(new_tree, filename='<ast>', mode='exec')

# 실행
print("=== 변환된 코드 실행 ===")
exec(code_obj)

print("\n=== 함수 호출 ===")
add(1, 2)
greet("World")

## 8. 연습 문제

아래 문제들을 직접 풀어보며 AST 다루기를 연습해보세요.

### 문제 1: AST 구조 분석

다음 코드의 AST 구조를 분석하고, 각 노드의 타입과 관계를 설명하세요.

In [ ]:
# 문제 1 코드
code = '''
def max_of_three(a, b, c):
    if a > b:
        if a > c:
            return a
        return c
    if b > c:
        return b
    return c
'''

# 여기에 코드를 작성하세요
tree = ast.parse(code)
print(ast.dump(tree, indent=2))

### 문제 2: 노드 카운터

AST의 각 노드 타입별 개수를 세는 함수를 작성하세요.

In [ ]:
from collections import Counter

def count_nodes(tree):
    """
    AST 트리의 각 노드 타입별 개수를 반환
    """
    counts = Counter()
    
    # 여기에 구현하세요
    def visit(node):
        counts[type(node).__name__] += 1
        for field, value in ast.iter_fields(node):
            if isinstance(value, list):
                for item in value:
                    if isinstance(item, ast.AST):
                        visit(item)
            elif isinstance(value, ast.AST):
                visit(value)
    
    visit(tree)
    return counts

# 테스트
code = '''
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)
'''

tree = ast.parse(code)
counts = count_nodes(tree)
print("노드 카운트:")
for node_type, count in counts.most_common():
    print(f"  {node_type}: {count}")

### 문제 3: 변수 이름 추출

코드에서 사용된 모든 변수 이름을 추출하는 함수를 작성하세요.

In [ ]:
def extract_variable_names(code):
    """
    코드에서 사용된 모든 변수 이름 추출
    """
    tree = ast.parse(code)
    names = set()
    
    # 여기에 구현하세요
    for node in ast.walk(tree):
        if isinstance(node, ast.Name):
            names.add(node.id)
        elif isinstance(node, ast.FunctionDef):
            names.add(node.name)
            for arg in node.args.args:
                names.add(arg.arg)
    
    return names

# 테스트
code = '''
def calculate(x, y):
    result = x + y
    temp = result * 2
    return temp
'''

names = extract_variable_names(code)
print(f"추출된 변수 이름: {sorted(names)}")

### 문제 4: 커스텀 Transformer

모든 `+` 연산을 `-` 연산으로 바꾸는 Transformer를 작성하세요.

In [ ]:
class AddToSubTransformer(ast.NodeTransformer):
    """
    모든 + 연산을 - 연산으로 변경
    """
    
    def visit_BinOp(self, node):
        # 여기에 구현하세요
        if isinstance(node.op, ast.Add):
            node.op = ast.Sub()
        return self.generic_visit(node)

# 테스트
code = "result = a + b + c"
tree = ast.parse(code)

transformer = AddToSubTransformer()
new_tree = transformer.visit(tree)

print("원본:", code)
print("\n변환된 AST:")
print(ast.dump(new_tree, indent=2))

# 실행 결과 확인
a, b, c = 10, 3, 2
code_obj = compile(new_tree, filename='<ast>', mode='exec')
exec(code_obj)
print(f"\n실행 결과: result = {result}")  # 10 - 3 - 2 = 5


## 정리

이 노트북에서 배운 내용:

1. **AST란?** - 소스 코드의 구조를 트리 형태로 표현한 것
2. **ast 모듈 기본 사용법** - `ast.parse()`, `ast.dump()`
3. **AST 노드 유형** - 문장(Statement)과 표현식(Expression) 노드
4. **AST 분석** - 트리 구조 탐색 및 노드 정보 추출
5. **AST 시각화** - `tools/ast_visualizer` 사용
6. **AST 변환** - `NodeTransformer`를 활용한 코드 수정

### 다음 단계

- Module 3: pdb를 활용한 디버깅 기법
- 실제 프로젝트에 AST 기반 코드 분석 적용